In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm, trange
import math
# files
import requests
from io import BytesIO
from zipfile import ZipFile
import os
import shutil
from pathlib import Path
# from scipy import interpolate
import matplotlib.pyplot as plt
# GIS
from osgeo import gdal
import rasterio
from affine import Affine
import richdem as rd
from rasterio.fill import fillnodata
# torch and dataset
import torch
import torchvision
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

# functions

from dataFunctions import *


In [2]:
path_ANH4 = '/Volumes/HydesT7/Grad Project/Data/AHN4_all/'
list_patch = []
with open(path_ANH4+'train_valid_patchs.txt') as fp:
    # list_patch.append(patch)
    list_patch = [line.rstrip('\n') for line in fp]

In [3]:
path_pre = '/Volumes/HydesT7/Grad Project/Data/AHN4_preprocessed_1_8/'
ls_dsm30 = []
ls_dtm30 = []
ls_dsm30_fill = []
ls_dsm30_Roughness = []
ls_dsm30_slope = []

for name in list_patch:
    ls_dsm30.append('R30_'+name+'.TIF')
    ls_dtm30.append('M30_'+name+'.TIF')
    ls_dsm30_fill.append('R30_'+name+'_filled.TIF')
    ls_dsm30_Roughness.append('R30_'+name+'_Roughness.TIF')
    ls_dsm30_slope.append('R30_'+name+'_slope.TIF')

In [5]:
rd.LoadGDAL(path_pre+ls_dsm30[0]).shape

(208, 167)

In [14]:
cut_length = 64
cut_width = 64

dsm_slices = []
dtm_slices = []
ground_truth_slices = []
dsm_fill_slices = []
dsm_fill_arrs = []
roughness_slices = []
roughness_arrs = []
slope_slices = []
slope_arrs = []

print("Slicing...")
for i in trange(len(list_patch)):
    dsm30 = readTIFF(path_pre, ls_dsm30[i], masked=True)
    dtm30 = readTIFF(path_pre, ls_dtm30[i], masked=True)
    dsm30_fill = readTIFF(path_pre, ls_dsm30_fill[i])
    dsm30_Roughness = readTIFF(path_pre, ls_dsm30_Roughness[i])
    dsm30_slope = readTIFF(path_pre, ls_dsm30_slope[i])

    dsm_fill_arrs.append(dsm30_fill)
    roughness_arrs.append(dsm30_Roughness)
    slope_arrs.append(dsm30_slope)

    dsm_slices += sliceRisPic(dsm30, cut_width, cut_length)
    dtm_slices += sliceRisPic(dtm30, cut_width, cut_length)
    dsm_fill_slices += sliceRisPic(dsm30_fill, cut_width, cut_length)
    roughness_slices += sliceRisPic(dsm30_Roughness, cut_width, cut_length)
    slope_slices += sliceRisPic(dsm30_slope, cut_width, cut_length)

mean_dsm, std_dsm = getNormalParams(dsm_fill_arrs)
mean_roughness, std_roughness = getNormalParams(roughness_arrs)
mean_slope, std_slope = getNormalParams(slope_arrs)

dsm_slices += augmentateData(dsm_slices)
dtm_slices += augmentateData(dtm_slices)
dsm_fill_slices += augmentateData(dsm_fill_slices)
roughness_slices += augmentateData(roughness_slices)
slope_slices += augmentateData(slope_slices)

print("Generating ground truth...")

for nn in trange(len(dsm_slices)):
    ground_truth_slices.append(
        generateGroundTruth(dsm_slices[nn], dtm_slices[nn], 1e-10))

Slicing...


100%|██████████| 455/455 [00:06<00:00, 68.75it/s]


Generating ground truth...


100%|██████████| 43680/43680 [00:36<00:00, 1209.06it/s]


In [15]:
para_means = np.asarray([mean_dsm, mean_roughness, mean_slope])
para_stds = np.asarray([std_dsm, std_roughness, std_slope])

display(para_means, para_stds)

array([9.90327775, 2.48690886, 1.35894729])

array([19.5961904 ,  2.92635182,  1.85182111])

In [16]:
Pics = stackPics(dsm_fill_slices, roughness_slices, slope_slices)
Masks = np.stack(ground_truth_slices)
Masks = Masks.astype(np.float32)
display(Pics.shape, Masks.shape)

(43680, 64, 64, 3)

(43680, 2, 64, 64)

In [17]:
out_file_path = '/Volumes/HydesT7/Grad Project/Data/U_Net/64_64_1_8_np_re/'
Path(out_file_path).mkdir(parents=True, exist_ok=True)
np.save(out_file_path+'Pics', Pics)
np.save(out_file_path+'Masks_1e-10', Masks)
np.save(out_file_path+'para_means', para_means)
np.save(out_file_path+'para_stds', para_stds)